In [ ]:
# Import necessary modules
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb
import logging

In [ ]:
# Enable debug logging
logging.basicConfig(level=logging.DEBUG)

# Define embedding function
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# Load documents
documents = SimpleDirectoryReader("./dataset").load_data()

# Step 1: Initialize Chroma with PersistentClient and create collection
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("llama_index_test_collection")

In [ ]:
# Step 2: Set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

# Verify that documents were added
print("Document count after initial load:", chroma_collection.count())


In [ ]:
# Step 3: Query data
from llama_index.llms.ollama import Ollama
query_engine = index.as_query_engine(llm=Ollama(model="llama3.2:1b-instruct-fp16", request_timeout=360.0))
response = query_engine.query("How to save money?")
display(Markdown(f"<b>{response}</b>"))